# 클로저 : 프로그래밍의 즐거움 Chapter 6. 지연과 불변성

## Younggun Na (amazingguni@gmail.com)

## Overview

- 불변성(immutability)
- 영속성 구조 설계하기(트리)
- 키워드의 용도(e.g., :keyword)
- 지연 퀵 정렬

## 6.1 불변성(immutability)

Programming language 차원에서 `불변성`를 추구하면 어려운 문제점들을 단순하게 해결할 수 있다.

#### 예정된 불변

모든 불변성 객체가 가질 수 있는 속성들은 **생성 시점에 정의되고 그 이후에는 변경될 수 없음**

> `예정론`: 전적으로 자유로운 것이란 존재하지 않고 모든 것은 계획되어 있다는 개념 -> 즉 한 번 만들어지면 뭘해도 바뀌지 않는다는 말...

#### 합의에 의한 불변

시스템 내에서 불변성이 전반적으로 잘 활용되려면 불변성에 대한 전체적인 약속이 필요

가령 *Java*에서 불변 클래스를 생성하려면

1. 클래스 자체와 그에 해당하는 모든 필드가 final로 선언되어야 함
2. 객체를 생성하는 과정에서 this를 통한 객체의 참조가 불가능함
3. 내부의 불변 객체들은 클래스 안에서 복사만 허용되고 외부로 참조가 되지 않아야 한다.

```java
final class A{
    final private int num;
    final private object obj;
    public A(num){
        this.num = num;
    }
    
    public object getObject(){
        return object.clone();
    }
}
```

클로저는 핵심적인 데이터 구조들을 통해 언어의 특징으로써 불변성을 직접 지원

- 불변성 구조를 만드는 것과 구현할 내용 각각에 의해 발생하는 복잡성을 분리
- 구현할 내용에 더욱 집중할 수 있다

### 6.1.2 불변성은 왜 사용하는가?

클로저의 불변성 데이터 구조는 단순히 언어 자체에 국한되는 것이 아니라, 언어의 핵심적인 철학에까지 연결되어 있다.

#### 불변(invariant)

불변성을 기반으로 하는 프로그래밍에서는 클래스나 함수에 대해 인스턴스가 특정 상태에 있을 때 값을 입력하려 하면 에러가 발생하도록 한다.

가변성 시스템에 불변성을 부여하려면 오직 생성에서만 값을 입력할 수 있도록 하여야 함

#### 객체 상태 예측

불변 객체의 운명은 이미 정해져 있기 때문에 예측이 쉽다.

#### 동일성의 의미

가변성의 세계에서는 동일성이 의미가 없음

- 두 객체가 현재 시점에서 동일하다고 해도 이후까지는 보장이 되지 않는다.
- 만일 두 객체가 동일하지 않다면,, 그들은 같다고 말할 수 없다.

``` java
class A{
    public int num;
    
    public A(int num){
        this.num = num;
    }
}
A a = new A(3); // 이 때는 a.num은 3이다.
a.num = 5; // 하지만.. a.num이 5로 변했다.


foo(a); // 인수로 전달되는 a의 속성은 전달 당시의 값과 같다고 보장되지 않는다.
```

불변성 객체는 지금 당장 동일하다면, 항상 동일할 것이라고 보장한다.

#### 쉬운 공유

어떤 객체가 절대 변경되지 않으리라는 확신이 있다면 **참조**를 제공함으로써 간편하게 공유할 수 있다.

* 만약 java에서 그렇게 하려면 방어적인 복사가 필요하다 (*Object.clone같은.*)

#### 간접성 감소

클로저에는 `가변성 참조`만 있다.

![title](6_var.png)

* Java에서는 기본적으로 가변성 데이터를 가리키는 참조가 있다.
* 이로 불필요한 복잡성이 크게 감소



#### 병행 프로그래밍을 뒷받침하는 불변성

불변성 객체는 항상 스레드에 안전하다.

* 변경으로 인한 에러에 구애받지 않고 다른 스레드에 자유롭게 전달할 수 있음

## 6.2 구조적 공유: 영속적 구조

리스트는 가장 단순한 형태의 *공유 구조* 타입

> 공유 구조에서는 변경되는 값을 위한 메모리만 새로 할당하고 변하지 않는 나머지 부분은 원래 객체에서 공유

In [5]:
(def l `(1 2 3))
(def m1 (rest l))
(def m2 (into `(4 5 6) l))
(def r (reverse m1))

(println l)
(println m1)
(println m2)
(println r)

(1 2 3)
(2 3)
(3 2 1 4 5 6)
(3 2)


#'user/l #'user/m1 #'user/m2 #'user/r nil nil nil nil

![title](6_clojure_list.gif)

In [14]:
(def mapv1 {:name "paul" :age 45})
(def mapv2 (assoc mapv1 :sec :male))

(println mapv1)
(println mapv2)

{:name paul, :age 45}
{:name paul, :age 45, :sec :male}


#'user/mapv1 #'user/mapv2 nil nil

항상 복사를 하면 효율적이지 않기 때문에 클로저의 컬렉션에서는 공유 구조를 가지고 있다.

In [3]:
(def baselist (list :barnabas :adam))
(def list1 (cons :willie baselist))
(def list2 (cons :phoenix baselist))

(println list1)
(println list2)

(:willie :barnabas :adam)
(:phoenix :barnabas :adam)


#'user/baselist #'user/list1 #'user/list2 nil nil

In [4]:
(= (next list1) (next list2))
(identical? (next list1) (next list2))

true true

#### 불변 객체로 이루어진 트리

여기서 만들어볼 트리는 Left Right Value를 가진 간단한 트리이다.

맵에 넣어보면 아래와 같다

In [8]:
{:val 5, :L nil, :R nil}

{:val 5, :L nil, :R nil}

In [ ]:
값으로 5를 가지고 왼쪽 가지가 비어있고 오른쪽 가지도 비어있는 하나의 노드

트리에 노드를 추가해주는 함수를 만들 수 있다.

In [6]:
(defn xconj [tree value]
  (cond
    (nil? tree) {:val value, :L nil, :R nil}))

#'user/xconj

In [7]:
(xconj nil 5)

{:val 5, :L nil, :R nil}

이제 첫 노드를 추가할 차례이다.

tree가 nil이 아니고 자신보다 value가 작은 경우에는 왼쪽 가지에 추가한다.

그를 위해 아래와 같은 비교가 필요해진다.

```clojure
(< value (:value tree))
```

또한, 아래와 같이 기존 노드의 값을 변경하지 않고 새로운 노드를 만들어서 복사해야 한다.

```clojure
(:value (:value tree),
  :L (여기에 새로운 노드를 입력)
  :R (:R tree))
```

#### 전체 코드

In [16]:
(defn xconj [tree value]
  (cond
    (nil? tree)           {:val value, :L nil, :R nil}
    (< value (:val tree)) {:val (:val tree),
                           :L   (xconj (:L tree) value)
                           :R   (:R tree)}
    :else                 {:val (:val tree),
                           :L   (:L tree)
                           :R   (xconj (:R tree) value)}
  )
)

#'user/xconj

In [17]:
(def tree1 (xconj nil 5))
(println tree1)
(def tree1 (xconj tree1 3))
(println tree1)
(def tree1 (xconj tree1 2))
(println tree1)

{:val 5, :L nil, :R nil}
{:val 5, :L {:val 3, :L nil, :R nil}, :R nil}
{:val 5, :L {:val 3, :L {:val 2, :L nil, :R nil}, :R nil}, :R nil}


#'user/tree1 nil #'user/tree1 nil #'user/tree1 nil

In [18]:
tree1

{:val 5, :L {:val 3, :L {:val 2, :L nil, :R nil}, :R nil}, :R nil}

데이터가 잘 들어갔는지 확인하기 위해 순서대로 조회하면서 출력해주는 함수를 만들어보자

In [19]:
(defn xseq [tree]
  (when tree
    (concat (xseq (:L tree)) [(:val tree)] (xseq (:R tree)))))

#'user/xseq

In [20]:
(xseq tree1)

(2 3 5)

In [21]:
(def tree2 (xconj tree1 7))
(xseq tree2)

#'user/tree2 (2 3 5 7)

![title](6_tree.jpg)

In [22]:
(identical? (:L tree1) (:L tree2))

true

이 예제는 모든 클로저의 영속적 컬렉션들이 공통적으로 갖는 몇가지 특징들을 보여준다.

1. 모든 *변경*은 일단 새로운 루트 노드를 생성하고, 새 값이 삽입될 위치의 트리 경로에 새 노드를 추가한다.
2. 값과 변경되지 않는 가지가 복사되는 것이 아니라, 기존의 노드에서 새 노드로 그 참조가 복사된다.
3. 스레드에 안전하다. 
  - *xconf*를 호출하기 전에 존재하던 객체들은 어떤 방법으로든 변경되지 않기 때문
  
하지만 위의 코드는 좋지 않은 품질의 코드다.

1. 이진 트리 방식
2. 숫자 외에 다른 값을 저장할 수 없다.
3. 트리의 깊이가 깊어지면 오버 플로그 발생 가능성이 있다.
4. *xseq*가 지연 시퀀스가 아닌 트리 전체를 복사하는 방식이다.
5. 비 균형적인 트리를 생성할 수 있다.

클로저는 메모리 공간 절약을 위해 지연 시퀀스라는 개념에 크게 의존하고 있는데,, 이건 다음에..

## 6.3 지연

Clojure는 필요한 순간에 시퀀스 타입을 평가할 수 있다.

```clojure
(- 13 (+ 2 2))
```

`(+ 2 2)`의 결과는 - 함수에게 호출되자 마자 전달된다.

이 절에서는 

- 불필요한 계산이 수행되는 문제를 해소하기 위해 지연을 어떻게 활용하는지
- 무한 시퀀스를 활용하는 방법에 대해
- delay와 force를 통해 지연 구조를 만드는 방법에 대해

알아본다.

### 6.3.1 논리곱을 사용한 지연

간단한 지연의 예시 - 에러가 발생하지 않도록 해주는 지연 평가의 장점을 보여주는 코드

```java
if (obj != null && obj.isWhatiz()) {
    // 코드들
}
```

**클로저 예시**

In [15]:
(defn if-chain [x y z]
  (if x
    (if y
      (if z
        (do
          (println "Made it!")
          :all-truthy)))))

#'user/if-chain

In [16]:
(if-chain () 42 true)

Made it!


:all-truthy

In [17]:
(if-chain true true false)

nil

In [24]:
(if ()
  (print "true")
  (print "false")
)

true

nil

동일한 기능을 *and 매크로* 로도 할 수 있다.

In [3]:
(defn and-chain [x y z]
  (and x y z (do (println "Made it!") :all-truthy))
  )
  
(and-chain () 42 true)

Made it!


#'user/and-chain :all-truthy

### 6.3.2 지연 시퀀스 생성 이해하기

시퀀스를 인자로 받아 중첩 구조를 생성하는 단순한 함수를 구현해보자

```clojure
(steps [1 2 3 4])
=> [1 [2 [3 [4 []]]]]
```

재귀로 구현하면,

In [6]:
(defn rec-step [[x & xs]]
  (if x
    [x (rec-step xs)]
  []))

#'user/rec-step

In [7]:
(rec-step [1 2 3 4])

[1 [2 [3 [4 []]]]]

하지만 큰 셋을 넣어 이 함수를 실행하면 *StackOverflow* 가 나고 만다

In [8]:
(rec-step (range 200000))

[] (java.lang.StackOverflowError)

LongRange.java:   113 clojure.lang.LongRange/first  
       RT.java:   942 clojure.lang.RT/nthFrom       
       RT.java:   897 clojure.lang.RT/nth           
            []:     1 user$rec_step/invokeStatic    
            []:     3 user$rec_step/invokeStatic    
            []:     1 user$rec_step/invoke          
            []:     3 user$rec_step/invokeStatic    
            []:     1 user$rec_step/invoke          
            []:     3 user$rec_step/invokeStatic    
            []:     1 user$rec_step/invoke          
            []:     3 user$rec_step/invokeStatic    
            []:     1 user$rec_step/invoke          
            []:     3 user$rec_step/invokeStatic    
            []:     1 user$rec_step/invoke          
            []:     3 user$rec_step/invokeStatic    
            []:     1 user$rec_step/invoke          
            []:     3 user$rec_step/invokeStatic    
            []:     1 user$rec_step/invoke          
           

; no value returned

그래서 지연 시퀀스가 필요하다.

**지연 시퀀스(lazy sequence)?**

시퀀스의 내용이 뒤늦게(Lazy) 평가되는 것이 가능한데, 이렇게 되면 요소의 값은 처음에는 존재하지 않다가 사용자가 그 요소에 접근할 때만 계산되어 생겨나는 것이다. 이 값들은 한 번반 계산되면 계속 유지되는데, 이렇게 레이지 시퀀스에서 요소에 접근하면서 값이 계산되는 것을 '실현'이라고 하고, 모든 값이 실현되면 '완전 실현'되었다고 한다.

lazy-seq은 함수에 지연을 적용할 수 있게 도와주며 사용할 때 아래 4가지 규칙을 지키는 것이 좋다.

1. 지연 시퀀스를 생성하는 표현식의 가장 바깥쪽에 lazy-seq 매크로 사용
2. 연산 중에 다른 시퀀스를 사용해야 할 경우가 있다면 next 대신 rest를 사용 --> 그렇지 않을 때는 next사용
3. 시퀀스를 처리할 때는 가능한 고차 함수를 사용 7장에서 다시 다룸
4. 시퀀스의 헤드를 붙들고 있지 말자

**고차 함수**

1. 인자로 한개 이상의 함수를 취한다.
2. 함수를 결과로 리턴한다.


In [23]:
(def very-lazy (-> (iterate #(do (print \.) (inc %)) 1)
                    rest rest rest))

..

#'user/very-lazy

In [24]:
(def less-lazy (-> (iterate #(do (print \.) (inc %)) 1)
                    next next next))

..

#'user/less-lazy

In [25]:
(println (first very-lazy))

.4


nil

In [26]:
(println (first less-lazy))

.4


nil

** 지연 시퀀스(*lazy seq*)와 *rest* 사용하기**

lazy-seq 매크로로 step 함수를 구현하면

In [28]:
(defn lz-rec-step [s]
  (lazy-seq
    (if (seq s)
      [(first s) (lz-rec-step (rest s))]
    [])
  )
)

#'user/lz-rec-step

In [29]:
(lz-rec-step [1 2 3 4])

(1 (2 (3 (4 ()))))

In [30]:
(class (lz-rec-step [1 2 3 4]))

clojure.lang.LazySeq

In [33]:
(dorun (lz-rec-step (range 200000)))

nil

In [34]:
(doall (lz-rec-step (range 200000)))

[] (java.lang.StackOverflowError)

PersistentHashMap.java:   415 clojure.lang.PersistentHashMap$ArrayNode/assoc  
PersistentHashMap.java:   142 clojure.lang.PersistentHashMap/assoc            
              Var.java:   323 clojure.lang.Var/pushThreadBindings             
              core.clj:  1837 clojure.core$push_thread_bindings/invokeStatic  
        core_print.clj:    42 clojure.core$print_sequential/invokeStatic      
        core_print.clj:   153 clojure.core$fn__6072/invokeStatic              
          MultiFn.java:   233 clojure.lang.MultiFn/invoke                     
              core.clj:  3572 clojure.core$pr_on/invokeStatic                 
              core.clj:  3566 clojure.core$pr_on/invoke                       
        core_print.clj:    59 clojure.core$print_sequential/invokeStatic      
        core_print.clj:   153 clojure.core$fn__6072/invokeStatic              
          MultiFn.java:   233 clojure.lang.MultiFn/invoke                     
              cor

; no value returned

In [38]:
(defn simple-range [i limit]
  (lazy-seq
    (when (< i limit)
      (cons i (simple-range (inc i) limit)))))
      
(simple-range 0 9)

#'user/simple-range (0 1 2 3 4 5 6 7 8)

lazy seq의 각 항목은 실체화 된 상태, 실체화되지 않은 상태로 존재한다.

- 실체화되지 않은 상태에서는 실체화에 필요한 인자가 없는 함수 또는 closure만을 갖고 있다.
- 호출되면 thunk의 리턴 값이 대신 저장되고 실체화된다.

![title](6_lazy_seq.png)

In [41]:
(defn random-ints [limit]
"Returns a lazy seq of random integers in the range [0,limit)."
  (lazy-seq
    (println "realizing random number")
    (cons (rand-int limit)
      (random-ints limit))))

#'user/random-ints

In [42]:
(def rands (take 10 (random-ints 50)))

#'user/rands

In [43]:
(first rands)

realizing random number


24

In [44]:
(first rands)

24

In [45]:
(second rands)

realizing random number


40

In [46]:
(nth rands 3)  

realizing random number
realizing random number


8

In [47]:
(count rands)

realizing random number
realizing random number
realizing random number
realizing random number
realizing random number
realizing random number


10

In [48]:
(count rands)     

10

### 6.3.3 헤드 무시하기

클로저에서 지연을 사용할 때 가장 큰 장점은 연산 중의 중간 결과 값들을 모두 실체화 하지 않도록 막아준다는 점

만일 함수 어딘가에 있는 시퀀스의 헤드를 붙들고 사용한다면 가비지 컬렉션에서 제외될 것이다.

- 이런 상황은 로컬에 바인딩해도 발생할 수 있다.

In [62]:
(let [r (range 1e9)]
  (first r)
  (last r))

Clojure: Unbalanced parentheses or kernel timed-out while processing form.

; no value returned

In [63]:
(let [r (range 1e9)]
  (last r)
  (first r)
)

Clojure: Unbalanced parentheses or kernel timed-out while processing form.

; no value returned

### 6.3.4 무한 시퀀스 활용하기

클로저의 시퀀스는 지연 평가되기 때문에 길이가 무한할 가능성도 있다.

In [65]:
(defn triangle [n]
  (/ (* n (+ n 1)) 2))

#'user/triangle

In [66]:
(triangle 10)

55

In [71]:
(map triangle (range 1 11))

(1 3 6 10 15 21 28 36 45 55)

무한 시퀀스를 정의해서 원하는 항목을 찾는 *질의* 를 하는 예시

In [72]:
(def tri-nums (map triangle (iterate inc 1)))

#'user/tri-nums

In [74]:
(take 10 tri-nums)

(1 3 6 10 15 21 28 36 45 55)

In [76]:
(take 10 (filter even? tri-nums))

(6 10 28 36 66 78 120 136 190 210)

In [77]:
(nth tri-nums 99)

5050

In [78]:
(nth tri-nums 0)

1

In [79]:
(double (reduce + (take 1000 (map / tri-nums))))

1.998001998001998

In [80]:
(take 2 (drop-while #(< % 10000) tri-nums))

(10011 10153)

**drop-while**

첫번째 인자의 조건을 만족하는 lazy sequnce를 drop

### 6.3.5 delay와 force 매크로

클로저 시퀀스는 지연 평가 되지만 다른 것들은 그렇지 않다.

- 대부분의 경우 클로저의 표현식은 필요한 시점보다는 함수에 전달되기 전에 평가

그래서 클로저에서는 call-by-need(요구에 의한 호출)을 구현하여 제공한다.

- *delay* 매크로는 *force* 함수를 사용해서 강제로 평가하기 전까지 표현식의 평가를 연기시킨다.
- 이를 통해 *호출자가 필요할 때* 표현식을 평가하여 사용할 수 있게 된다.

In [84]:
(defn defer-expensive [cheap expensive]
  (if-let [good-enough (force cheap)]
    good-enough
    (force expensive)))

#'user/defer-expensive

In [85]:
(defer-expensive (delay :cheep) (delay (do (Thread/sleep 5000) :expensive)))

:cheep

In [87]:
(defer-expensive (delay false) (delay (do (Thread/sleep 1000) :expensive)))

:expensive

delay는 지연된 연산을 명시적으로 확인하기 좋고, 계산을 캐시하기 때문에 평가를 단 한 번만 한다.


** if-let이란 **

```clojure
(if :truthy-thing
  (let [res :truthy-thing] (println res)))
; :truthy-thing
```

동일한 문법이다.

```clojure
(if-let [res :truthy-thing] (println res))
; :truthy-thing
```

In [88]:
(defn inf-triangles [n]
  {:head (triangle n)
   :tail (delay (inf-triangles (inc n)))})

#'user/inf-triangles

In [90]:
(defn head [l] (:head l))
(defn tail [l] (force (:tail l)))

#'user/head #'user/tail

In [91]:
(def tri-nums (inf-triangles 1))

#'user/tri-nums 1

In [92]:
(head tri-nums)

1

In [93]:
(head (tail tri-nums))

3

In [95]:
(head (tail (tail tri-nums)))

6

![title](6_lazy_linked_list.png)

## 6.4 종합하기: 지연 퀵 정렬

여기서는 다른 방법들과 다른 방법으로 퀵 정렬을 구현한다.

1. 지연을 활용한 꼬리 재귀 버전의 퀵 정렬
2. 점진적 실행의 문제로부터 분리시킨다. 시퀀스에서 오직 계산에 필요한 부분만 계산되어야 한다.

### 6.4.1 구현

정렬할 seq를 생성하기 위해 *rand-ints* 함수를 먼저 구현한다.

In [4]:
(defn rand-ints [n]
  (take n (repeatedly #(rand-int n))))
  
(rand-ints 10)

#'user/rand-ints (5 7 6 5 6 5 2 3 7 7)

In [8]:
(defn print-seq [s]
  (let [[part & parts] (list s)]
  (println part parts))
  )
  
  
(print-seq (rand-ints 10))

(5 9 6 9 9 9 7 4 9 1) nil


#'user/print-seq nil

In [38]:
(defn sort-parts [work]
  (lazy-seq
    (loop [[part & parts] work]
      (println "-------------------")
      (println "part" part)
      (println "parts" parts)
      (if-let [[pivot & xs] (seq part)]
          (let [smaller? #(< % pivot)]
                (println (list*
                        (filter smaller? xs)
                        pivot
                        (remove smaller? xs)
                        parts))
                (recur (list*
                        (filter smaller? xs)
                        pivot
                        (remove smaller? xs)
                        parts)))
      (when-let [[x & parts] parts]
          (cons x (sort-parts parts)))))))
          
(defn qsort [xs]
  (sort-parts (list xs)))

#'user/sort-parts #'user/qsort

In [39]:
(qsort [2 1 4 3])

-------------------
part [2 1 4 3]
parts nil
((1) 2 (4 3))
-------------------
part (1)
parts (2 (4 3))
(() 1 () 2 (4 3))
-------------------
part ()
parts (1 () 2 (4 3))
-------------------
part ()
parts (2 (4 3))
-------------------
part (4 3)
parts nil
((3) 4 ())
-------------------
part (3)
parts (4 ())
(() 3 () 4 ())
-------------------
part ()
parts (3 () 4 ())
-------------------
part ()
parts (4 ())
-------------------
part ()
parts nil


(1 2 3 4)

![title](6_quick.jpg)

In [36]:
(qsort [5 3 1 7 4 2 8 6])

-------------------
part [5 3 1 7 4 2 8 6]
parts nil
-------------------
part (3 1 4 2)
parts (5 (7 8 6))
-------------------
part (1 2)
parts (3 (4) 5 (7 8 6))
-------------------
part ()
parts (1 (2) 3 (4) 5 (7 8 6))
-------------------
part (2)
parts (3 (4) 5 (7 8 6))
-------------------
part ()
parts (2 () 3 (4) 5 (7 8 6))
-------------------
part ()
parts (3 (4) 5 (7 8 6))
-------------------
part (4)
parts (5 (7 8 6))
-------------------
part ()
parts (4 () 5 (7 8 6))
-------------------
part ()
parts (5 (7 8 6))
-------------------
part (7 8 6)
parts nil
-------------------
part (6)
parts (7 (8))
-------------------
part ()
parts (6 () 7 (8))
-------------------
part ()
parts (7 (8))
-------------------
part (8)
parts nil
-------------------
part ()
parts (8 ())
-------------------
part ()
parts nil


(1 2 3 4 5 6 7 8)

In [40]:
(take 10 (qsort (rand-ints 10000)))

-------------------
part (9400 4487 5644 4932 7812 7994 2277 6968 8310 4901 2664 6461 9254 6603 9930 9267 733 5039 6413 2489 3987 4897 4088 4975 395 2694 2086 9787 9530 2996 8045 2731 56 5145 101 1206 5558 3507 5428 9306 9793 478 9415 8364 2326 4283 6802 6920 5996 2919 2393 522 2261 1539 4336 9643 9115 3003 7366 3562 8967 398 6914 9072 630 5889 7513 5535 428 2438 568 1979 2284 3654 7195 7702 5643 2306 1620 5121 3067 9878 9373 2545 6459 7977 2418 4918 1964 7336 256 2656 8864 1723 2042 2182 6980 2138 9789 6981 492 1362 6864 4602 3437 109 9133 3346 3601 3859 8358 2172 7074 3750 8766 8712 5275 5850 8021 3269 9923 591 3329 1118 9124 8159 1804 1560 9510 6267 1860 9736 5219 3871 8930 6452 4349 8112 4542 2000 7537 2052 2483 2258 1269 7631 9603 9253 662 1656 1652 3803 2559 2627 6497 4843 9 2716 8007 8633 3457 2338 4001 1130 2319 4377 698 8551 2972 1386 4357 4345 5696 1839 7954 5659 3566 863 7103 8735 928 6793 4766 600 7685 7370 8703 3958 8363 8400 9141 3207 1669 3343 6641 2160 7094 4405 1802 82

(0 1 2 5 6 6 7 7 9 10)

**&** 
이게 들어간 이후에는 가변인자가 들어온다

## 6.5 정리

- 클로저의 주요 데이터 타입들은 모두 기본적으로 불변성과 영속성을 가지고 있다.
- 이진 트리에서는 구조 공유를 통해 전체를 복사하지 않고 동작하도록 구현했다.
- 하지만 이는 메모리의 효율성을 보장하지 않기 때문에 지연의 장점을 활용하였다.
- 지연과 꼬리 재귀(recur)를 활용한 퀵정렬의 구현으로 시퀀스 전체가 메모리상에서 실제화되지 않도록 보장한다는 것을 배움


